# **EDA**

In [1]:
# import tarfile
# tf = tarfile.open('RetailDemoDataSet.tar')
# tf.extractall("data")

In [1]:
import pandas as pd
import json
import numpy as np
import time
from datetime import datetime
import matplotlib.pyplot as plt

In [19]:
items = pd.read_csv('movies.csv')
items_df = items.rename(columns={'movieId':'ITEM_ID',
                              'title' : 'ITEM_NAME',
                              'genres' :'CATEGORY_L1',
                             })
print(items_df.shape)
items_df.head()

(58098, 3)


,ITEM_ID,ITEM_NAME,CATEGORY_L1
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [12]:
# users = pd.read_csv('users.csv')
# user_cols = ['id', 'username', 'age', 'gender']
# users_df = users[user_cols]
# users_df = users_df.rename(columns={'id':'USER_ID',
#                               'username' : 'USER_NAME',
#                               'age' :'AGE',
#                               'gender' : 'GENDER',                              
#                              })

# users_df['USER_ID'] = users_df['USER_ID'].astype('object')
# print(users_df.shape)
# print(users_df.dtypes)
# users_df.head()

In [20]:
its = pd.read_csv('ratings.csv')
its_df = its.rename(columns={'userId': 'USER_ID',
                            'movieId':'ITEM_ID',
                              'rating' : 'EVENT_TYPE',
                              'timestamp' :'TIMESTAMP',
                             })
its_df['USER_ID'] = its_df['USER_ID'].astype('object')
print(its_df.shape)
its_df.head()

(994882, 4)


,USER_ID,ITEM_ID,EVENT_TYPE,TIMESTAMP
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264


In [21]:
#merge1 = pd.merge(its_df, users_df, on='USER_ID', how='left')
df = pd.merge(its_df, items_df, on='ITEM_ID', how='left')
df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'], unit='s')

In [22]:
print(df.shape)
df.head()

(994882, 6)


,USER_ID,ITEM_ID,EVENT_TYPE,TIMESTAMP,ITEM_NAME,CATEGORY_L1
0,1,307,3.5,2009-10-27 21:00:21,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
1,1,481,3.5,2009-10-27 21:04:16,Kalifornia (1993),Drama|Thriller
2,1,1091,1.5,2009-10-27 21:04:31,Weekend at Bernie's (1989),Comedy
3,1,1257,4.5,2009-10-27 21:04:20,Better Off Dead... (1985),Comedy|Romance
4,1,1449,4.5,2009-10-27 21:01:04,Waiting for Guffman (1996),Comedy


In [23]:
df.to_csv('interaction_data.csv', index=False)

In [24]:
pd.options.display.max_rows = 5
def split_holdout(data, pct):
    df = data.copy()
    # Rank per each subgroup, 'USER_ID'
    ranks = df.groupby('USER_ID').TIMESTAMP.rank(pct=True, method='first')
    df = df.join((ranks> pct).to_frame('holdout'))
    
    holdout = df[df['holdout']].drop('holdout', axis=1)
    train = df[~df['holdout']].drop('holdout', axis=1)    
    
    return train, holdout

df_warm_train, df_warm_holdout = split_holdout(df, pct=0.9)

In [25]:
print(df_warm_train.shape)
print(df_warm_holdout.shape)

(890817, 6)
(104065, 6)


In [26]:
df.shape

(994882, 6)

In [27]:
df_warm_train

,USER_ID,ITEM_ID,EVENT_TYPE,TIMESTAMP,ITEM_NAME,CATEGORY_L1
0,1,307,3.5,2009-10-27 21:00:21,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
1,1,481,3.5,2009-10-27 21:04:16,Kalifornia (1993),Drama|Thriller
...,...,...,...,...,...,...
994856,10000,175485,2.5,2017-09-05 19:15:05,Death Note (2017),Horror|Thriller
994867,10000,178667,3.0,2017-10-06 00:00:45,Gerald's Game (2017),Horror|Thriller


In [28]:
df_warm_holdout

,USER_ID,ITEM_ID,EVENT_TYPE,TIMESTAMP,ITEM_NAME,CATEGORY_L1
9,1,2840,3.0,2009-10-27 21:05:00,Stigmata (1999),Drama|Thriller
10,1,2986,2.5,2009-10-27 21:04:56,RoboCop 2 (1990),Action|Crime|Sci-Fi|Thriller
...,...,...,...,...,...,...
994880,10000,185029,2.5,2018-08-10 17:34:01,A Quiet Place (2018),Drama|Horror|Thriller
994881,10000,185989,2.5,2018-07-10 09:57:10,Truth or Dare (2018),Horror|Thriller
